## Assignment 3

### Image mosaic

#### On grading

The homework is graded on a scale of 0 to 100, and each question has a specified number of points. If your answer is not entirely correct, but shows that you are on the right track, we may award partial credit. Overall, you need to acquire at least *45 points* to pass. Otherwise, it is considered as a fail.

In [ ]:
%matplotlib inline
from skimage import io, color, transform, img_as_ubyte, img_as_float
from skimage.io import imshow
import skimage
from skimage.transform import rescale
import numpy as np
import matplotlib.pyplot as plt

# Make figures larger
plt.rcParams['figure.figsize'] = (10.0, 8.0)

In this assignment, you will create your own photographic mosaic. A photographic mosaic is an image made up of tiny but complete images that replace the pixels or small patches of another base image. In simpler terms, the image comprises other images. It's a charming and meta concept that has gained immense popularity on the internet. People have, for instance, created an image of a character from [Harry Potter](http://orig04.deviantart.net/d3f5/f/2010/320/7/4/deathy_hallows_snape___mosaic_by_smallrinilady-d330420.jpg) made out of frames from the movies. 

Let's first clarify the terms we will be using in this assignment. The main image that serves as the base of the design is called the *background image*. For instance, in the example given above, the background image would be that of Severus Snape. The smaller images that are used to create the background image are called *replacement images* as they replace the different parts of the background image. In our example, the frames of the movies would be the replacement images. 

The creation of the mosaic is split into two tasks:

1. **Make tiles**. Divide the background image into tiles. You can think of each tile as a separate, smaller image.
2. **Image similarity search**. For each tile, find the most similar image from the collection of replacement images and substitute the tile.

## Make tiles

### Ex. 1.1: Reading the file and shaping the image (20 points)

Begin by reading in the background image from the file `kangaroo.jpg` and store a reference in a variable called `bg_image`. As you have probably guessed it is not Harry Potter.

<img src="kangaroo.jpg" /> 

Our tiles will be $32 \times 32$ pixels. This turns out to be a convenient choice, because our pool of replacement images are also $32 \times 32$ pixels, and therefore no resizing will be necessary.

Here are some clearer instructions to read in the background image from the file `kangaroo.jpg` and store it in a variable named `bg_image`:

1. Load the `kangaroo.jpg` file as the background image using an appropriate function from `skimage`.
2. Convert the integer representation of the image to a float representation using an appropriate function provided by the `skimage` library. This step ensures that the color values are in the range of [0, 1].
3. Rescale the image to $120\%$ of its original size using an appropriate function. 
4. Adjust the shape of `bg_image` to fit exactly a whole number of tiles, which are 32 by 32 pixels each. Here, you should discard any excess rows and columns without scaling the image. From now on, we work on this adjusted image whose number of rows and columns are divisible by $32$. 

In [ ]:
tile_size = 32
# Your code here

fig, axs = plt.subplots(3)

bg_image = io.imread("kangaroo.jpg")
bg_image = img_as_float(bg_image)

scale_factor = 1.2
bg_image_rescale = rescale(bg_image, scale_factor, channel_axis=2)

tile_size = 32
rows, cols, channels = bg_image_rescale.shape

adjusted_rows = (rows // tile_size) * tile_size
adjusted_cols = (cols // tile_size) * tile_size

bg_image = bg_image_rescale[:adjusted_rows, :adjusted_cols, :]

axs[0].imshow(io.imread("kangaroo.jpg"))
axs[0].set_title("Original Image")
axs[0].axis('off')


axs[1].imshow(bg_image_rescale)  
axs[1].set_title("Rescaled Image")
axs[1].axis('off')


axs[2].imshow(bg_image)  
axs[2].set_title("Adjusted Image (32x32 tiles)")
axs[2].axis('off')
for r in range(0, adjusted_rows, tile_size):
    plt.axhline(r, color='white', linestyle='-', linewidth=1)
for c in range(0, adjusted_cols, tile_size):
    plt.axvline(c, color='white', linestyle='-', linewidth=1)



plt.tight_layout()
plt.show()

bg_image.shape


### Ex. 1.2: Tile coordinates (24 points)

When arranging tiles, we place them in rows and columns, similar to pixels. This allows us to identify the pixels that a tile at a specific row and column spans. 

The following code requires the completion of the `tile_coordinates` function. This function should return two separate 1-dimensional arrays - the first providing the row coordinates of the given tile, and the second providing the column coordinates.

These arrays are then used to index the pixels of the background image as shown here:

```
    ii, jj = tile_coordinates(i, j, tile_size) # it returns the row and column indices spanning the tile at position (i,j)
    tile = bg_image[ii, jj] # Indexing the pixel of the tile (i,j)   
```

In order to prove the effectiveness of the process, you can utilize `tile_coordinates` function to generate a new image with lower resolution. In this image, every tile of the background image will be replaced with the *mean color* of that tile. Although the new image will have the same number of pixels as the background image, it will have a much lower resolution appearance, because each tile has a uniform color.

Complete the function `low_resolution` and test it with the `bg_image` and different tile sizes (`tile_size`).

In [ ]:
bg_image.shape

In [ ]:
def tile_coordinates(i, j, tile_size):
    # Your code here
    row_coords = np.arange(i * tile_size, (i + 1) * tile_size)
    
    col_coords = np.arange(j * tile_size, (j + 1) * tile_size)
    
    return row_coords, col_coords
    
def low_resolution(img, tile_size):
    # Your code here
    low_res = np.zeros_like(img)
    
    
    num_tiles_y = img.shape[0] // tile_size  
    num_tiles_x = img.shape[1] // tile_size  
    
    
    for i in range(num_tiles_y):
        for j in range(num_tiles_x):
            
            row_coords, col_coords = tile_coordinates(i, j, tile_size)
            
            tile = img[np.ix_(row_coords, col_coords)]
            
            mean_color = tile.mean(axis=(0, 1))
            
            low_res[np.ix_(row_coords, col_coords)] = mean_color
            
    return low_res

fig, ax = plt.subplots(4)
plt.subplots_adjust(wspace=0.5, hspace=.5)
for i,ts in enumerate([4,16,32,64]):
    ax[i].imshow(low_resolution(bg_image,ts))
    ax[i].set_title(f"ts={ts}")
plt.show()

## Image similarity search

After finishing the previous assignment, you have all the necessary components to create a photographic mosaic. The next step is to select images from a pool of replacements most similar to the tiles they are replacing.

To begin, you need to load the set of replacement images.

### Ex. 1.3: Load the replacement images (10 points)

To access the CIFAR-10 dataset, navigate to the dataset homepage <a href=http://www.cs.toronto.edu/~kriz/cifar.html>here</a>. Once there, locate and click the download link specifically for the Python version of the dataset. Verify that you have selected the correct edition before proceeding with the download. The download will result in a compressed file named `cifar-10-python.tar.gz`. Store this file in the same directory as your notebook file. Use your preferred file extraction tool to decompress the file. Once decompressed, the directory containing your notebook should now display a new subdirectory named `cifar-10-batches-py`, which contains the dataset's contents.

In the `cifar` module, we have included a `load_cifar10` function. You can use this function to load image data from a file. It accepts a filename as a parameter and returns a 4-dimensional `numpy` array that contains the image data. Additionally, it also returns a 1-dimensional array that holds the labels for each image. These labels indicate the category of the image. While the labels are not mandatory for this assignment, you can use them to filter images based on your preferences. For example, you can use them to work with images of a particular category, like kittens.

You can use the images from only one batch, for example, `data_batch_1`, but feel free.

In [ ]:
from cifar import load_cifar10
replacements, replacement_labels = load_cifar10("cifar-10-batches-py/data_batch_1")


Display a random image from the collection. Your code should display a new random image each time the cell is executed.

In [ ]:
# Your code here

random_index = np.random.randint(0, len(replacements))

random_image = replacements[random_index]

plt.imshow(random_image)
plt.title(f'Label: {replacement_labels[random_index]}')
plt.axis('off')
plt.show()

### Image search

Let's focus on a single tile to understand how to find a similar replacement image. The process is the same for all the tiles. The basic idea is to represent both the tile and possible replacement images as vectors and measure the distance between them. If the distance between a tile and a replacement image is low, it means that they are similar, and the replacement image is a good candidate. The best candidate is the one that minimizes the distance.

One way to approach finding replacement images is to think of it as an image search engine. You provide a query image and an image collection, and in return, you get a ranked list of the most similar images from the collection. We have provided two functions that implement parts of this process. The `image_distance_query` function calculates the distance between query images and all images in a collection, and the `plot_best_matches` function visualizes the query images with the top k matches.

However, there is one crucial ingredient missing, and the quality of image retrieval depends entirely on it. We need a function that can convert raw images into vectors that can be compared. Below, we will ask you to write several versions of this function. Please note that you should not have to change anything in the cell following this text.

In [ ]:
from scipy.spatial.distance import cdist

def image_distance_query(query_images, collection_images, to_vector_fn, p=2):
    query_vecs = [to_vector_fn(image).reshape(1, -1) for image in query_images]
    query_vecs = np.concatenate(query_vecs)

    collection_vecs = [to_vector_fn(image).reshape(1, -1) for image in collection_images]
    collection_vecs = np.concatenate(collection_vecs)

    return cdist(query_vecs, collection_vecs, 'minkowski', p=p)

def plot_best_matches(distances, query_images, collection_images, top_k=5):
    fig, axes = plt.subplots(nrows=distances.shape[0], ncols=top_k+1, squeeze=False,
                             figsize=(top_k*2, distances.shape[0]*2))

    for i in range(distances.shape[0]):
        top_indices = distances[i].argsort().ravel()
        for j in range(top_k):
            ax = axes[i, j+1]
            ax.axis('off')
            plt.sca(ax)
            top_index = top_indices[j]
            imshow(collection_images[top_index])
            ax.set_title("Top " + str(j + 1) + " match")
        
        ax = axes[i, 0]
        ax.set_title("Query image")
        ax.axis('off')
        plt.sca(ax)
        imshow(query_images[i])
        


### Ex 1.4: Average color vector (20 points)

The `image_distance_query` function needs a function argument called `to_vector_fn` that converts a 2-dimensional image to a 1-dimensional array or vector. Note that the size of the resulting vector is independent of the input image size. The quality of image retrieval depends on how you define this function. 

To start with, we need to complete the implementation of the `average_color` function, which returns the mean RGB color across all pixels. It means that the vector representation of an input image is an array of size tree corresponding to the mean values for red, green, and blue over all pixels. 

To calculate distances and visualize the results, you can use the `image_distance_query` and `plot_best_matches` functions. Choose 5 images from the replacement pool as query images and use a subset of the replacement images (e.g. 1000) while developing your code. However, make sure to switch to the full set before submitting your work.

In [ ]:
def average_color(image_rgb):
    # Your code here
    
    return np.array(image_rgb.mean(axis=(0,1)))


# Your code here
query_images = [replacements[0], replacements[1], replacements[2], replacements[3], replacements[4]]  # 5 query images
collection_images = replacements[:1000]

distances = image_distance_query(query_images, collection_images, average_color)

plot_best_matches(distances, query_images, collection_images)
plt.show()


### (OPT) Ex 1.5: 8 by 8 grayscale

Create a new function that first reduces the images to grayscale, then resizes the $32 \times 32$ input image to an $8 \times 8$ image by taking the mean pixel intensity in all $4 \times 4$ squares. (You can check if there is a function in `skimage` that does this). Finally flatten the image to one dimension before returning.

Calculate distances and visualize as before.

In [ ]:

def scale_image(image, shape=(8,8)):
    nr, nc = shape
    rimg = np.zeros((nr, nc))
    
    # Your code here
    return rimg

# Your code here



### (OPT) Ex 1.6: Hue histogram

Create a function that converts an image to the HSV color space using `color.rgb2hsv`. The function should then create a histogram on the hue (first) channel. It's important to note that the edges of the bins should be fixed to ensure that they are comparable across different images.

Once the histogram is created, calculate the distances and visualize them as you have done before.

In [ ]:
# Your code here
def hsv(image):
    hsv_image = color.rgb2hsv(image)

    # Your code here

    return hist

# Your code here

### (OPT) Ex 1.7: Joint hue and saturation histogram

Extend the function from the previous assigment to create a joint histogram over the hue and saturation channels. As before the bins should be fixed. Use the function `np.histogram2d` to create the histograms.

In [ ]:
# Your code here
def extended_hsv(image):
    hsv_image = color.rgb2hsv(image)

    # Your code here
    return hist

# Your code here

### (OPT) Ex 1.8: Any other strategy you might think of

Here is a chance to be creative. Mixing strategies is also a possiblity.

In [ ]:
# Your code here (possibly)

### Ex 1.9: Render photographic mosaic (26 points)

Take the best image-to-vector function from above and use it together with `image_distance_query` to build your image mosaic. You may reuse some of the code you wrote in the `low_resolution` function. 

For computational efficiency you should make sure to only call `image_distance_query` once, and not inside of the loop!

OPTIONAL: As an optional exercise, write your code in a way that it can process different values for `tile_size`. 

In [ ]:
# Your code here


def create_mosaic(bg_image, collection_images, tile_size):
    
    
    tile_rows = bg_image.shape[0] // tile_size
    tile_cols = bg_image.shape[1] // tile_size
    
    tiles_v = []
    for row in range(tile_rows):
        for col in range(tile_cols):
            i_coords, j_coords = tile_coordinates(row, col, tile_size)
            tile_pixels = bg_image[np.ix_(i_coords, j_coords)]
            tiles_v.append(tile_pixels)

    tiles_v = np.array(tiles_v)
    tiles = tiles_v
    distances = image_distance_query(tiles, collection_images, average_color)
    
    mosaic = np.zeros_like(bg_image)
    num_tiles_y = bg_image.shape[0] // tile_size
    num_tiles_x = bg_image.shape[1] // tile_size
    
    for i in range(num_tiles_y):
        for j in range(num_tiles_x):
            tile_index = i * num_tiles_x + j
            closest_image_index = np.argmin(distances[tile_index])
            closest_image = collection_images[closest_image_index]
            resized_image = transform.resize(
            closest_image, 
            (tile_size, tile_size, bg_image.shape[2]), 
            mode='reflect', 
            anti_aliasing=True, 
            preserve_range=True
        ).astype(bg_image.dtype)
            mosaic[
                i * tile_size : (i + 1) * tile_size,
                j * tile_size : (j + 1) * tile_size
            ] = resized_image
    
    return mosaic



tile_size = 32
mosaic = create_mosaic(bg_image, collection_images, tile_size)


plt.imshow(mosaic)
plt.axis('off')
plt.show()


### Image collections

Here is a small list of free image sources in case you would like to use other (or larger) images for your mosaic.

- [CIFAR-10](http://www.vision.ee.ethz.ch/datasets_extra/pec/). The CIFAR-10 dataset consists of 60,000 32x32 colour images in 10 classes, with 6000 images per class. 
- [Food-101](http://www.vision.ee.ethz.ch/datasets_extra/food-101/). We introduce a challenging data set of 101 food categories, with 101,000 images. All images were rescaled to have a maximum side length of 512 pixels.
- [MIRFLICKR-25000](http://press.liacs.nl/mirflickr/mirdownload.html). 25,000 images from Flickr. 